In this notebook, our objective is to import the data we have prepared in the previous notebook into our Postgres-based Data Warehouse.

In [1]:
# import libraries
import pandas as pd
import psycopg2

In [2]:
df = pd.read_csv("../data/final_data.csv")

Our database, named "bank_reviews_dw," consists of three tables: "date," "reviews," and "bank."

The "date" table is a dimension table that stores information about the date when a review was made. It has columns for the "id_date" (a unique identifier for each date), "year," "month," and "day" values.

The "reviews" table is a fact table that contains details about the reviews. It includes the "reviewId" as the primary key, representing the unique identifier for each review. The "sentiment" column stores the sentiment score, "stars" represents the rating given to the bank, and "category" stores the category of the sentiment.

The "bank" table is another dimension table that holds information about the banks. It includes the "id_bank" as the primary key, "cid" for the bank identifier, "bank" for the bank name, "country" for the country name, "city" for the city where the bank is located, "rank" for the bank's ranking, "lat" for the latitude, and "lng" for the longitude.

In [3]:
# Establish a connection to your PostgreSQL database
conn = psycopg2.connect(database="bank_reviews_dw", user="postgres", password="yasir", host="localhost", port="5432")
cur = conn.cursor()
# Create the Date table (Dimension Table)
cur.execute('''
    CREATE TABLE IF NOT EXISTS date (
        id_date SERIAL PRIMARY KEY,
        year INTEGER,
        month INTEGER,
        day INTEGER
    )
''')
# Create the Reviews table (Fact Table)
cur.execute('''
    CREATE TABLE IF NOT EXISTS reviews (
        id_reviwe SERIAL PRIMARY KEY,
        reviewId VARCHAR(255),
        sentiment FLOAT,
        stars FLOAT,
        category VARCHAR(255)
    )
''')

# Create the Bank table (Dimension Table)
cur.execute('''
    CREATE TABLE IF NOT EXISTS bank (
        id_bank SERIAL PRIMARY KEY,
        cid VARCHAR(255) ,
        bank VARCHAR(255),
        country VARCHAR(255),
        city VARCHAR(255),
        rank FLOAT,
        lat FLOAT,
        lng FLOAT
    )
''')

# Insert data into the date table
for _, row in df.iterrows():
    cur.execute("INSERT INTO date (year, month, day) VALUES (%s, %s, %s)",
                (row['year'], row['month'], row['day']))

# Insert data into the reviews table
for _, row in df.iterrows():
    cur.execute("INSERT INTO reviews (reviewId, sentiment, stars, category) VALUES (%s, %s, %s, %s)",
                (row['reviewId'], row['sentiment'], row['stars'], row['sentiment_category']))

# Insert data into the bank table
for _, row in df.iterrows():
    cur.execute("INSERT INTO bank (cid, bank, country, city, rank, lat, lng) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (row['cid'], row['bank'], row['country'] , row['city'], row['rank'], row['lat'], row['lng']))

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()